In [137]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

results = pd.read_csv("data_raw/results.csv")
races = pd.read_csv("data_raw/races.csv")
circuits = pd.read_csv("data_raw/circuits.csv")


 Introduction
 A markdown text with 1-2 paragraph that summarize the main goals of the project. The first paragraph should briefly describe what Formula 1 is, what question you’re interested in, and why it is relevant. The introduction should end with a high-level description of the results and the coming structure of the project. Try to make the text self-contained, intended for someone who isn’t familiar with Formula 1 or the dataset

Data description: 
Write a markdown chunk of 1 paragraph describing which dataset tables (among the 14) you will be using. State what each row represents, how many observations are contained in each table, the years, and a brief overview of the of the data that is contained there:
▪ Import any necessary libraries.
▪ Import the data.
▪ Do any calculations for counting the number of rows, etc.


In [138]:
# Get the number of rows in each dataset
results_num_rows = results.shape[0]
races_num_rows = races.shape[0]
circuits_num_rows = circuits.shape[0]

print(f"The number of rows in the results dataset is {results_num_rows}.")
print(f"The number of rows in the races dataset is {races_num_rows}.")
print(f"The number of rows in the circuits dataset is {circuits_num_rows}.")

The number of rows in the results dataset is 25840.
The number of rows in the races dataset is 1102.
The number of rows in the circuits dataset is 77.


In [139]:
unique_years = races['year'].unique()
print(unique_years)

# Get the number of unique release years in the merged dataset
num_unique_years = races['year'].nunique()

# Print the resulting number
print(num_unique_years)

[2009 2008 2007 2006 2005 2004 2003 2002 2001 2000 1999 1998 1997 1996
 1995 1994 1993 1992 1991 1990 2010 1989 1988 1987 1986 1985 1984 1983
 1982 1981 1980 1979 1978 1977 1976 1975 1974 1973 1972 1971 1970 1969
 1968 1967 1966 1965 1964 1963 1962 1961 1960 1959 1958 1957 1956 1955
 1954 1953 1952 1951 1950 2011 2012 2013 2014 2015 2016 2017 2018 2019
 2020 2021 2022 2023]
74


o Write a paragraph in markdown summarizing data cleaning procedures: 
▪ Include code for data cleaning.

In [141]:
# clean alt
circuits["alt"].str.isnumeric()
subset      = circuits.query("alt.str.isnumeric() == False")
list_unique = pd.unique(subset["alt"])
print(list_unique)
list_old = ['\\N','-7']
list_new = [np.nan,-7]
circuits["alt"] = circuits["alt"].replace(list_old, list_new)
circuits["alt_numeric"] = pd.to_numeric(circuits["alt"])


# clean fastestlapspeed
results["fastestLapSpeed"].str.isnumeric()
subset2      = results.query("fastestLapSpeed.str.isnumeric() == False")
list_unique = pd.unique(subset2["fastestLapSpeed"])
print(list_unique)
results['fastestLapSpeed'] = results['fastestLapSpeed'].replace('\\N', np.nan)
results["fastestLapSpeed_numeric"] = pd.to_numeric(results["fastestLapSpeed"])

fastestLapSpeed_numeric

[]
['218.300' '217.586' '216.719' ... '211.632' '208.556' '209.889']


o Write a paragraph in markdown describing any merging procedures: 
▪ Include code for merging. 

In [142]:
# merge
results["fastestLapSpeed_numeric"].mean()
race_with_alt_clean = pd.merge(races,
                      circuits[["circuitId","alt_numeric"]],
                        on = "circuitId",
                        how = "left")


data_merge   = pd.merge(race_with_alt_clean,
                      results[["raceId","fastestLapSpeed_numeric"]],
                        on = "raceId",
                        how = "left")

In [143]:
# Get the number of rows in the merged dataset
num_rows = data_merge.shape[0]

# Print the resulting number
print(num_rows)

25863


o Write a paragraph describing your main columns:
▪ Compute a table of descriptive statistics for the main columns of the merged dataset that you’re interested. Try to be selective. The idea is to do a deeper analysis of a few columns rather than to do a lot.

In [146]:
bins_alt = [-1000,0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 900, 1000, 10000]
labels_alt = ["less than 0",
            "Between 0 and 50",
            "Between 50 and 100",
            "Between 100 and 150",
            "Between 150 and 200",
            "Between 200 and 250",
            "Between 250 and 300",
            "Between 300 and 350",
            "Between 350 and 400",
            "Between 400 and 450",
            "Between 450 and 500",
            "Between 500 and 550",
            "Between 550 and 600",
            "Between 600 and 650",
            "Between 650 and 700",
            "Between 700 and 750",
            "Between 750 and 800",
            "Between 800 and 900",
            "Between 900 and 1000",
            "greater than 1000"]
           

data_merge["bins_alt"] = pd.cut(data_merge["alt_numeric"],
                              bins = bins_alt,
                              right = True,
                              labels = labels_alt)



## descriptive statistics/result table
table = data_merge.groupby("bins_alt").agg(mean_fastestLapSpeed = ('fastestLapSpeed_numeric', 'mean'),
                                           median_fastestLapSpeed = ('fastestLapSpeed_numeric', 'median'),
                                           std_fastestLapSpeed = ('fastestLapSpeed_numeric', 'std'))
table

,mean_fastestLapSpeed,median_fastestLapSpeed,std_fastestLapSpeed
bins_alt,,,
less than 0,197.247349,199.6540,8.804440
Between 0 and 50,195.402612,200.5935,21.580479
Between 50 and 100,NaN,NaN,NaN
Between 100 and 150,204.482128,206.5600,11.341499
Between 150 and 200,223.523929,226.9060,19.603777
Between 200 and 250,203.902916,204.5680,9.142384
Between 250 and 300,188.403188,189.2690,12.300952
Between 300 and 350,NaN,NaN,NaN
Between 350 and 400,NaN,NaN,NaN


Results:
o This should contain a combination of code to produce tables/plots and
markdown text explaining what the findings are.
o Be creative! The idea is to understand the relationship between different sets of
columns to answer an interesting question about the data.

In [ ]:
plt.bar(labels_alt, table["mean_fastestLapSpeed"])

Discussion:
o Provide a brief 1 paragraph markdown chunk summarizing your findings. Describe the main things you learned from the data